# 1 Bibliotecas Python

## 1.1 Instalações

In [3]:
#pip install pandas

In [4]:
#pip install inflection

## 1.2 Importações

In [5]:
import pandas as pd
import inflection
import math

## 1.3 Configurações

In [6]:
# Suprimir notação científica no Pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# 2 Coleta de Dados

## 2.1 Leitura

In [7]:
df_vendas_cru = pd.read_csv('../dados/rossmann-store-sales/train.csv')
df_lojas_cru = pd.read_csv('../dados/rossmann-store-sales/store.csv')

/home/mg/Documents/data_science_em_producao/dsproducao/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df_vendas_cru.head(3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1


In [9]:
df_lojas_cru.head(3)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.00,9.00,2008.00,0,NaN,NaN,NaN
1,2,a,a,570.00,11.00,2007.00,1,13.00,2010.00,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.00,12.00,2006.00,1,14.00,2011.00,"Jan,Apr,Jul,Oct"


In [10]:
df_cru = pd.merge(df_vendas_cru, df_lojas_cru, how='inner', on='Store')

In [11]:
df_cru.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
899881,987,7,2014-03-09,0,0,0,0,0,0,c,a,1690.00,6.00,2007.00,0,NaN,NaN,NaN


# 3 Descrição dos Dados

In [12]:
df1 = df_cru.copy()

## 3.1 Renomeando colunas

In [13]:
colunas_originais = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 
                     'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 
                     'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 
                     'Promo2SinceYear', 'PromoInterval']
# Renomeando as colunas para o padrão snacke case
snackecase = lambda x: inflection.underscore(x)
novas_colunas = list(map(snackecase, colunas_originais))
df1.columns = novas_colunas

## 3.2 Dimensão dos dados

In [14]:
print(f'Número de linhas: {df1.shape[0]}')
print(f'Número de colunas: {df1.shape[1]}')

Número de linhas: 1017209
Número de colunas: 18


## 3.3 Tipos de dados

In [15]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 3.4 Dados faltantes

In [16]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 3.5 Tratamento dos NaN

### competition_distance              

Representa a distância do concorrente mais próximo

Suposição: 
1. Não possuem concorrentes próximos; 
2. A distância com relação ao concorrente é muito longa.

Tratamento: 
1. Preencher as linhas com um registro maior do que a distância máxima registrada na coluna.

In [17]:
print(f"Maior distância do concorrente para a loja: {df1['competition_distance'].max()}")
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)
print(f"Dados NaN restantes: {df1['competition_distance'].isna().sum()}")                 

Maior distância do concorrente para a loja: 75860.0
Dados NaN restantes: 0


### competition_open_since_month

Representa aproximadamente o mês que o concorrente mais próximo abriu a loja

Suposição: 
1. Falta do concorrente próximo;
2. O concorrente próximo abriu a loja antes;
3. A data de abertura do concorrente não foi registrada. 

Tratamento:
1. Extrair da coluna date o mês da data da venda, por considerar que as vendas são influenciadas pela data no qual o concorrente próximo abriu. 

In [18]:
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

In [19]:
print(f'Dados faltantes na coluna: {df1["competition_open_since_month"].isna().sum()}')

Dados faltantes na coluna: 0


### competition_open_since_year

Representa aproximadamente o ano que o concorrente mais próximo abriu a loja

Suposição: 
1. Falta do concorrente próximo;
2. O concorrente próximo abriu a loja antes;
3. A data de abertura do concorrente não foi registrada. 

Tratamento:
1. Extrair da coluna date o ano da data da venda, por considerar que as vendas são influenciadas pela data no qual o concorrente próximo abriu. 

In [20]:
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

In [21]:
print(f'Dados faltantes na coluna: {df1["competition_open_since_year"].isna().sum()}')

Dados faltantes na coluna: 0


### promo2_since_week

In [22]:
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

In [24]:
print(f'Dados faltantes na coluna: {df1["promo2_since_week"].isna().sum()}')

Dados faltantes na coluna: 0


### promo2_since_year 

In [25]:
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

In [26]:
print(f'Dados faltantes na coluna: {df1["promo2_since_year"].isna().sum()}')

Dados faltantes na coluna: 0


### promo_interval

In [50]:
month_map = {1: 'Jan',
            2: 'Feb',
            3: 'Mar',
            4: 'Apr',
            5: 'May',
            6: 'Jun',
            7: 'Jul',
            8: 'Aug',
            9: 'Sep',
            10: 'Oct',
            11: 'Nov',
            12: 'Dec'}

df1['promo_interval'].fillna(0, inplace=True)
df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [54]:
print(f'Dados faltantes na coluna: {df1["promo_interval"].isna().sum()}')

Dados faltantes na coluna: 0


## 3.6

## 3.7

## 3.8